In [32]:
import sqlite3

In [43]:
sqlite_connection = sqlite3.connect('database.db')

In [44]:
cursor = sqlite_connection.cursor()

### 1. Вывести всех людей, записанных на собеседование 25.06

Вывести ФИО этих людей, их номера телефона и вакансии, на которые они подаются (join)

In [35]:
query = """
select day, time, vacancies.name, applicants.name, applicants.number from calendar
join applicants on applicants.id = calendar.applicant_id
join vacancies on vacancies.id = calendar.vacancy_id
where calendar.day = 25.06
"""

In [36]:
res = cursor.execute(query)
res.fetchall()

[(25.06,
  '10:00',
  'Менеджер по подбору персонала',
  'Иванов Иван Иванович',
  '+79995553535'),
 (25.06,
  '11:00',
  'Администратор (м. Ясенево)',
  'Иванов Иван Иванович',
  '+79995553535'),
 (25.06,
  '18:00',
  'Повар горячего цеха',
  'Сидорова Елена Александровна',
  '+79998553535')]

### 2. Вывести, сколько собеседований у каждого человека в день

Вывести день, ФИО и количество собеседований (агрегация)

In [49]:
query = """
select calendar.day, applicants.name, count(*) from calendar
join applicants on applicants.id = calendar.applicant_id
join vacancies on vacancies.id = calendar.vacancy_id
GROUP BY calendar.day, applicants.id
"""

In [50]:
res = cursor.execute(query)
res.fetchall()

[(25.06, 'Сидорова Елена Александровна', 1),
 (25.06, 'Иванов Иван Иванович', 2),
 (26.06, 'Сидорова Елена Александровна', 1)]

### 3. Вывести всю информацию о вакансиях, на которые есть собеседования

(вложенный select)

In [55]:
query = """
select * from vacancies
where vacancies.id in (
        select vacancy_id from calendar
    )
"""

In [56]:
res = cursor.execute(query)
res.fetchall()

[(43068111,
  'Повар горячего цеха',
  'Санкт-Петербург',
  0,
  'опыт работы на позициях: повар-универсал, повар холодного цеха, повар горячего цеха, повар открытой кухни, повар-кулинар от 6 месяцев;\nсреднее профессиональное образование (училище, колледж, техникум) или высшее приветствуется;\nдоброжелательность, энергичность, аккуратность;\nжелание и умение работать в команде;\nналичие санитарной книжки (с действительными сроками медицинского обследования) или готовность ее сделать.',
  'оформление согласно ТК РФ;\nполный социальный пакет;\nбелая заработная плата: оклад 42432 руб. (за 180 часов) + премии + бонусы - НДФЛ;\nкомпенсация проезда 3741 руб. и бонус за некурение 500 руб. с первого месяца;\nвозможность повысить категорию и доход уже с первого месяца работы;\nграфик работы: 2/2 или 3/3 по 12 часов;\nвыплаты заработной платы 3 раза в месяц;\nкорпоративное питание, стильная униформа;\nДМС через 1 год работы в Компании;\nсофинансирование занятий спортом;\nобучение, возможности к

### 4. Вывести дату и время собеседования в Москве с людьми старше 20

(Where с объединением трёх таблиц в where)

In [71]:
query = """
select day, time from calendar
where calendar.vacancy_id in (
        select id from vacancies
        where city = "Москва"
    ) and
    calendar.applicant_id in (
        select id from applicants
        where age > 20
    )
"""

In [72]:
res = cursor.execute(query)
res.fetchall()

[(25.06, '10:00'), (25.06, '11:00')]

### 5. CRUD

Добавить новое собеседование (Update)

In [74]:
update = """
update calendar
set applicant_id = 0, vacancy_id = 43068111
where day = 25.06 and time = "15:00"
"""

In [75]:
cursor.execute(update)

In [76]:
sqlite_connection.commit()

Убрать нерелевантную вакансию (Delete)

In [78]:
cursor.execute("DELETE FROM vacancies WHERE id = 45507457")
sqlite_connection.commit()

### Triggers

In [80]:
cursor.execute(
    '''CREATE TRIGGER offer
             AFTER INSERT ON applicants
             when new.age < 18
             BEGIN
                 SELECT RAISE (ABORT, 'underage!');
             END
             ;
             ''')

OperationalError: trigger offer already exists

In [81]:
cursor.execute("INSERT INTO applicants (name, city, age) VALUES ('Миша', 'Москва', 15)")
sqlite_connection.commit()

IntegrityError: underage!